In [1]:
from IPython.display import Image

# 4 좋은 훈련 데이터셋 만들기 - 데이터 전처리

# 4.1 누락된 데이터 다루기

### 4.1.1 테이블 형태 데이터에서 누락된 값 식별

In [2]:
import pandas as pd
from io import StringIO

csv_data = \
'''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0,'''

df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


* 누락값 = NaN

In [3]:
# isnull : 셀이 수치 값을 담고 있는지(False) 또는 누락되어 있는지(True)를 나타내는 불리언 값이 채워진 DataFrame 반환
df.isnull()

,A,B,C,D
0,False,False,False,False
1,False,False,True,False
2,False,False,False,True


In [4]:
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

### 4.1.2 누락된 값이 있는 훈련 샘플이나 특성 제외
* 해당 훈련 샘플(행)이나 특성(열)(을 완전히 삭제
    * dropna : 누락값이 있는 행(axis=0) 또는 열(axis=1) 삭제

In [5]:
df.dropna(axis=0)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [6]:
df.dropna(axis=1)

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


In [7]:
# 모든 열이 NaN인 행 삭제
df.dropna(how='all')

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [8]:
# NaN 아닌 값이 4개보다 작은 행을 삭제
df.dropna(thresh=4)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [9]:
# 특정 열에 NaN이 있는 행만 삭제
df.dropna(subset=['C'])

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,10.0,11.0,12.0,NaN


### 4.1.3 누락된 값 대체
* 평균
* 중간값
* 가장 많이 나타난 값
    * 사이킷런의 SimpleImputer 클래스 사용

In [10]:
from sklearn.impute import SimpleImputer
import numpy as np

imr = SimpleImputer(missing_values=np.nan, strategy='mean') # mean : 평균, median : 중간값, most_frequent : 가장 많이 나타난 값
imr = imr.fit(df.values)
imputed_data = imr.transform(df.values)
imputed_data

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

In [11]:
# strategy='contant' 일때 fill_value 매개변수에 채우려는 값 지정
# SimpleInputer : axis=0. -> 열 방향
# FunctionTransformer : axis=1 -> 행 방향
from sklearn.preprocessing import FunctionTransformer

ftr_imr = FunctionTransformer(lambda x: imr.fit_transform(x.T).T, validate=False)
imputed_data = ftr_imr.fit_transform(df.values)
imputed_data

array([[ 1.        ,  2.        ,  3.        ,  4.        ],
       [ 5.        ,  6.        ,  6.33333333,  8.        ],
       [10.        , 11.        , 12.        , 11.        ]])

In [12]:
# SimpleInputer 클래스의 add_indicator 매개변수를 True로 지정하면 indicator_속성이 추가되고
# transform 메서드가 누락된 값의 위치를 포함된 배열을 반환
imr = SimpleImputer(add_indicator=True)
imputed_data = imr.fit_transform(df.values)
imputed_data

array([[ 1. ,  2. ,  3. ,  4. ,  0. ,  0. ],
       [ 5. ,  6. ,  7.5,  8. ,  1. ,  0. ],
       [10. , 11. , 12. ,  6. ,  0. ,  1. ]])

In [13]:
# 추가된 indicator_ 속성은 MissingIndicator 클래스의 객체
imr.indicator_

MissingIndicator(error_on_new=False)

In [14]:
# MissingIndicator 객체의 features_ 속성은 누락된 값이 있는 특성의 인덱스를 담고 있음
imr.indicator_.features_

array([2, 3])

In [15]:
# MissingIndicator 객체의 fit_transform 메서드를 호출하면 features_ 속성에 담긴 특성에서 누락된 값의 위치를 나타내는 배열 반환
imr.indicator_.fit_transform(df.values)

array([[False, False],
       [ True, False],
       [False,  True]])

In [16]:
# inverse_transform 메서드로 원본 특성으로 변환
imr.inverse_transform(imputed_data)

array([[ 1.,  2.,  3.,  4.],
       [ 5.,  6., nan,  8.],
       [10., 11., 12., nan]])

* SimpleImputer는 한 특성의 통계값을 사용하여 누락값을 채움.
* IterativeImputer 클래스
    * 다른 특성을 사용하여 누락된 값을 예측
    1. initial_strategy 매개변수에 지정된 방식으로 누락된 값 초기화
    2. 누락된 값이 있는 한 특성을 타깃으로 삼고 다른 특성을 사용해 모델을 훈련하여 예측
    * 모든 특성 순회
    * initial_strategy : 'mean', 'median', 'most_frequent' ,'constant'
    * 예측할 특성을 선택하는 순서(default=ascending)
        * ascending : 누락된 값이 가장 적은 특성부터 선택
        * descending : 누락된 값이 가장 큰 특성부터 선택
        * roman : 왼쪽에서 오른쪽으로 선택
        * arabic : 오른쪽에서 왼쪽으로 선택
        * random : 랜덤하게 고름
    * 특성 예측은 종료 조건을 만족할 때까지 반복
        * 반복 시 이전 단계와 절대값 차이 중 가장 큰 값이 누락된 값을 제외하고 가장 큰 절대값에 tol 매개변수를 곱한 것 보다 작을 경우 종료
            * tol 기본값 = 1e-3
        * max_iter 매개변수에서 지정한 횟수에 도달하면 종료

예측에 사용하는 모델은 estimator 매개변수에 지정할 수 있으며, 기본적으로 BayesianRidge 클래스 사용. 예측에 사용할 특성 개수는 n_nearest_features에서 지정할 수 있으며 상관계수가 높은 특성을 우선하게 랜덤하게 선택. default=None, 모든 특성 사용

In [17]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

iimr = IterativeImputer()
iimr.fit_transform(df.values)

array([[ 1.        ,  2.        ,  3.        ,  4.        ],
       [ 5.        ,  6.        ,  7.00047063,  8.        ],
       [10.        , 11.        , 12.        , 12.99964527]])

In [18]:
# KNNImputer 클래스는 K-최근접 이웃 방법을 사용해 누락값 대체.
# n_neighbors : 최근접 이웃 개수, default=5
# 샘플 개수가 n_neighbors보다 작으면 SimpleImputer(strategy='mean')과 같은 결과
from sklearn.impute import KNNImputer

kimr = KNNImputer()
kimr.fit_transform(df.values)

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

In [19]:
# fillna : 누락값 채귀
# methid = 'bfill' 또는 'backfill' => 누락된 값을 다음 행의 값으로 채움
# method = 'ffill' 또는 'pad' => 누락된 값을 이전 행의 값으로 채움
df.fillna(method='bfill')

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,12.0,8.0
2,10.0,11.0,12.0,NaN


In [20]:
df.fillna(method='backfill')

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,12.0,8.0
2,10.0,11.0,12.0,NaN


In [21]:
df.fillna(method='ffill')

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,3.0,8.0
2,10.0,11.0,12.0,8.0


In [22]:
df.fillna(method='pad')

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,3.0,8.0
2,10.0,11.0,12.0,8.0


In [23]:
# axis=1을 지정하면 열을 사용
df.fillna(method='ffill', axis=1) # 이전 열의 값으로 누락된 값 채움

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,6.0,8.0
2,10.0,11.0,12.0,12.0


### 4.1.4 사이킷런 추정기 API 익히기
* SimpleImputer 클래스는 데이터 변환에 사용되는 사이킷런의 변환기(transformer) 클래스
    * 주요 메서드
        * fit : 훈련 데이터에서 모델 파라미터를 학습
        * transform : 학습한 파라미터로 데이터 변환
    * 변환하려는 데이터 배열은 모델 학습에 사용한 데이터의 특성 개수와 같아야한다.

In [24]:
# 사이킷런 변환기의 훈련과 변환 과정
Image(url='https://git.io/Jtmwv', width=400)

* 사이킷런 추정기(estimator) - classifier, regressor
    * 주요 파라미터
        * 추정기에는 predict 메서드가 있지만, transform 메서드도 가질 수 있다.

In [25]:
# 사이킷런 추정기의 훈련과 예측 과정
Image(url='https://git.io/JtYZW', width=300)

# 4.2 범주형 데이터 다루기

* 순서가 있는 것
    * 정렬하거나 차례대로 놓을 수 있는 범주형 특성

* 순서가 없는 것
    * 차례를 부여할 수 없음

### 4.2.1 판다스를 사용한 범주형 데이터 인코딩

In [26]:
df = pd.DataFrame([['green', 'M', 10.1, 'class2'],
                   ['red', 'L', 13.5, 'class1'],
                   ['blue', 'XL', 15.3, 'class2']])

df.columns = ['color', 'size', 'price', 'classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class2
1,red,L,13.5,class1
2,blue,XL,15.3,class2


### 4.2.2 순서가 있는 특성 매핑
* 학습 알고리즘이 순서 특성을 인식하려면 범주형의 문자열 값을 정수로 바꾸어야 한다.

In [27]:
size_mapping = {'XL':3, 'L':2, 'M':1}
df['size'] = df['size'].map(size_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class2
1,red,2,13.5,class1
2,blue,3,15.3,class2


In [28]:
# 원래대로 매핑
inv_size_mapping = {v: k for k, v in size_mapping.items()}
df['size'].map(inv_size_mapping)

0     M
1     L
2    XL
Name: size, dtype: object

### 4.2.3 클래스 레이블 인코딩
* 클래스 레이블을 정수로 변환

In [29]:
class_mapping = {label:idx for idx, label in enumerate(np.unique(df['classlabel']))}
class_mapping

{'class1': 0, 'class2': 1}

In [30]:
# 클래스 레이블을 문자열에서 정수로 바꿈
df['classlabel'] = df['classlabel'].map(class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,1
1,red,2,13.5,0
2,blue,3,15.3,1


In [31]:
# 클래스 레이블을 거꾸로 매핑합니다
inv_class_mapping = {v: k for k, v in class_mapping.items()}
df['classlabel'] = df['classlabel'].map(inv_class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class2
1,red,2,13.5,class1
2,blue,3,15.3,class2


In [32]:
# 사이킷런의 LabelEncoder 클래스
from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder()
y = class_le.fit_transform(df['classlabel'].values) # fit_transform : fit + transform
y

array([1, 0, 1])

In [33]:
# inverse_transform 메서드를 사용하면 정수 클래스 레이블을 원본 문자열 형태로 되돌릴 수 있다
class_le.inverse_transform(y)

array(['class2', 'class1', 'class2'], dtype=object)

### 4.2.4 순서가 없는 특성에 원-핫 인코딩 적용


In [34]:
X = df[['color', 'size', 'price']].values
color_le = LabelEncoder()
X[:, 0] = color_le.fit_transform(X[:, 0])
X

array([[1, 1, 10.1],
       [2, 2, 13.5],
       [0, 3, 15.3]], dtype=object)

* blue = 0
* green = 1
* red = 2

* LabelEncoder : 타깃 레이블을 인코딩하기 위한 클래스이므로 입력 데이터로 1차원 배열을 넣는다. => 데이터셋에 변경해야 할 열이 많으면 번거롭다.
* OrdinalEncoder : 범주형  데이터를 정수로 인코딩
* ColumnTransformer : 판다스 데이터프레임의 열마다 다른 변환을 적용하도록 도와주어 여러 개의 열을 한번에 정수로 변환 가능

In [35]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
ord_enc = OrdinalEncoder(dtype=np.int)
col_trans = ColumnTransformer([('ord_enc', ord_enc, ['color'])])
X_trans = col_trans.fit_transform(df)
X_trans

array([[1],
       [2],
       [0]])

In [36]:
col_trans.named_transformers_['ord_enc'].inverse_transform(X_trans)

array([['green'],
       ['red'],
       ['blue']], dtype=object)

컬러값에는 어떠한 순서가 없지만, 학습 알고리즘이 green은 blue보다 크고 red는 green보다 크다고 가정할 것이다. 가정이 옳지는 않지만 알고리즘이 의미있는 결과는 만들 수 있다. 하지만 이 결과가 최선은 아니다. 그래서 이를 해결하기 위한 통상적인 방법이 원-핫 인코딩(one-hot encoding)이다.

* 원-핫 인코딩 : 순서 없는 특성에 들어 있는 고유한 값마다 새로운 더미 특성을 만드는 것

In [37]:
from sklearn.preprocessing import OneHotEncoder
X = df[['color', 'size', 'price']].values
color_ohe = OneHotEncoder()
color_ohe.fit_transform(X[:, 0].reshape(-1, 1)).toarray()

array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [38]:
# 여러 개의 특성이 있는 배열에서 특정 열만 변환 => ColumnTransformer
from sklearn.compose import ColumnTransformer
X = df[['color', 'size', 'price']].values
c_transf = ColumnTransformer([
    ('onehot', OneHotEncoder(), [0]), # (name, transformer, column(s))
    ('nothing', 'passthrough', [1, 2]) # 나머지 두 열은 변경하지 않고 그대로
])
c_transf.fit_transform(X)

array([[0.0, 1.0, 0.0, 1, 10.1],
       [0.0, 0.0, 1.0, 2, 13.5],
       [1.0, 0.0, 0.0, 3, 15.3]], dtype=object)

In [39]:
# 원-핫 이코딩으로 더미 변수를 만드는 더 편리한 방법은 판다스의 get_dummies 메서드를 사용하는 것.
# DataFrame에 적용하면 get_dummies 메서드는 문자열 열만 반환 나머지 열은 그대로 둔다
pd.get_dummies(df[['price','color', 'size']])

,price,size,color_blue,color_green,color_red
0,10.1,1,0,1,0
1,13.5,2,0,0,1
2,15.3,3,1,0,0


In [40]:
# get_dummies 메서드에서 columns 매개변수를 사용하면 변환하려는 특성을 구체적으로 지정할 수 있다
pd.get_dummies(df[['price','color', 'size']], columns=['size'])

,price,color,size_1,size_2,size_3
0,10.1,green,1,0,0
1,13.5,red,0,1,0
2,15.3,blue,0,0,1


* 원-핫 인코딩된 데이터셋 - 다중 공선성(multicollinearity) 문제
    * 어떤 알고리즘에는 이슈가 될 수 있다.
        * ex, 역행렬을 구해야하는 경우
    * 특성 간의 상관관계가 높으면 역행렬을 계산하기 어려워 수치적으로 불안정해진다.

변수 간의 상관관계를 감소하려면 원-핫 인코딩된 배열에서 특성 열 하나를 삭제한다. 특성을 삭제해도 잃는 정보는 없다.


In [41]:
# get_dummies에서 다중 공선성 문제 처리
pd.get_dummies(df[['price', 'color', 'size']], drop_first=True)

,price,size,color_green,color_red
0,10.1,1,1,0
1,13.5,2,0,1
2,15.3,3,0,0


OnehotEncoder에서 중복된 열을 삭제하려면 drop='first'와 categories='auto'로 지정.

In [42]:
# OneHotEncoder에서 다중 공선성 문제 처리
color_ohe = OneHotEncoder(categories='auto', drop='first')
c_transf = ColumnTransformer([('onehot', color_ohe, [0]),
                              ('nothing', 'passthrough', [1, 2])])
c_transf.fit_transform(X)

array([[1.0, 0.0, 1, 10.1],
       [0.0, 1.0, 2, 13.5],
       [0.0, 0.0, 3, 15.3]], dtype=object)

* 순서가 있는 특성 인코딩
    * 순서가 있는 특성의 범주 사이에서 수치적 크기에 대해 확신이 없거나 두 범수 사이의 순서를 정의할 수 없을 경우 => **임계값을 사용하여 0/1로 인코딩**

In [43]:
# M, L, XL 값을 가진 특성 size를 두 개의 새로운 특성 'x > M'과 'x > L'로 나눌 수 있다
df = pd.DataFrame([['green', 'M', 10.1, 'class2'],
                   ['red', 'L', 13.5, 'class1'],
                   ['blue', 'XL', 15.3, 'class2']])

df.columns = ['color', 'size', 'price', 'classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class2
1,red,L,13.5,class1
2,blue,XL,15.3,class2


In [44]:
# 판다스 데이터프레임의 apply 메서드를 사용하여 임계 값 기준으로 특성을 인코딩하는 lambda 함수를 적용할 수 있다.
df['x > M'] = df['size'].apply(lambda x: 1 if x in {'L', 'XL'} else 0)
df['x > L'] = df['size'].apply(lambda x: 1 if x == 'XL' else 0)
del df['size']
df

,color,price,classlabel,x > M,x > L
0,green,10.1,class2,0,0
1,red,13.5,class1,1,0
2,blue,15.3,class2,1,1


# 4.3 데이터셋을 훈련 데이터셋과 테스트 데이터셋으로 나누기
* 모델을 실전에 투입하기 전 테스트 데이터셋에 있는 레이블과 예측을 비교
    * 편향되지 않은 성능을 측성

* 데이터셋 : https://archive.ics.uci.edu/ml/datasets/Wine

In [45]:
df_wine = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data', header=None)
df_wine.columns = ['Class label', 'Alcohol', 'Malic acid', 'Ash',
                   'Alcalinity of ash', 'Magnesium', 'Total phenols',
                   'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins',
                   'Color intensity', 'Hue', 'OD280/OD315 of diluted wines',
                   'Proline']
print('Class labels', np.unique(df_wine['Class label']))
df_wine.head()

Class labels [1 2 3]


,Class label,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [46]:
df_wine.shape

(178, 14)

* 샘플은 1, 2, 3 세 개의 클래스 중 하나에 속해 있다.


In [47]:
# 훈련 데이터셋과 테스트 데이터셋 나누기
from sklearn.model_selection import train_test_split

X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)